In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
pd.options.mode.chained_assignment = None  

# trim data
def get_truncated_df(merge_df, columns, year_column_name=None, low=0.01, high=0.99):
    merge_df_copy = merge_df.copy()
    if year_column_name == None:
        for column in columns: 
            _1pct, _99pct = merge_df_copy[column].quantile(q=low), merge_df_copy[column].quantile(q=high)
            merge_df_copy[column].where((merge_df_copy[column] < _99pct) & (merge_df_copy[column] > _1pct), math.nan, inplace=True)
    else: 
        merge_df_list = []
        merge_year_df_list = [[year, merge_year_df] for year, merge_year_df in merge_df_copy.groupby(year_column_name)]
        for year, merge_year_df in tqdm(merge_year_df_list):
            for column in columns: 
                _1pct, _99pct = merge_year_df[column].quantile(q=low), merge_year_df[column].quantile(q=high)
                merge_year_df[column].where(~((merge_year_df[column] > _99pct) | (merge_year_df[column] < _1pct)), math.nan, inplace=True)
            merge_df_list.append(merge_year_df)
        merge_df_copy = pd.concat(merge_df_list)
    
    return merge_df_copy

def get_winsorized_df(merge_df, columns, year_column_name=None, low=0.01, high=0.99):
    merge_df_copy = merge_df.copy()
    if year_column_name == None:
        for column in columns: 
            _1pct, _99pct = merge_df_copy[column].quantile(q=low), merge_df_copy[column].quantile(q=high)
            merge_df_copy[column].where((merge_df_copy[column] < _99pct), _99pct, inplace=True)
            merge_df_copy[column].where((merge_df_copy[column] > _1pct), _1pct, inplace=True)
    else: 
        merge_df_list = []
        merge_year_df_list = [[year, merge_year_df] for year, merge_year_df in merge_df_copy.groupby(year_column_name)]
        for year, merge_year_df in tqdm(merge_year_df_list):
            for column in columns: 
                low_val, high_val = merge_year_df[column].quantile(q=low), merge_year_df[column].quantile(q=high)
                merge_year_df[column].where(~(merge_year_df[column] > high_val), high_val, inplace=True)
                merge_year_df[column].where(~(merge_year_df[column] < low_val), low_val, inplace=True)
            merge_df_list.append(merge_year_df)
        merge_df_copy = pd.concat(merge_df_list)
    return merge_df_copy

def get_cal_qtr(pmon):
    if pmon in [1, 2, 3]: 
        return 1 
    elif pmon in [4, 5, 6]: 
        return 2
    elif pmon in [7, 8, 9]: 
        return 3
    else: 
        return 4
    
def winsorize_strict(x, low, high): 
    if x < low: 
        return low 
    elif x > high: 
        return high
    else: 
        return x

def trim_strict(x, low, high): 
    if x < low: 
        return math.nan 
    elif x > high: 
        return math.nan
    else: 
        return x
    
def get_2d_sic(x): 
    try: 
        return float(str(x)[:2])
    except: 
        return math.nan
    
def trim_bouchaud(col, df):
    sue_prc_stats = df[col].describe()
    iqr, med = (sue_prc_stats.loc['75%'] - sue_prc_stats.loc['25%']), sue_prc_stats.loc['50%']
    delta = 5 * iqr
    return df[col].apply(lambda x: trim_strict(x, low=med-delta, high=med+delta))

def trim_bouchaud_cols(cols, df):
    df = df.copy(deep=True)
    for col in tqdm(cols):
        df[col] = trim_bouchaud(col, df)
    return df

## Load Data

In [2]:
## read cleaned IBES file
sue_df = pd.read_pickle('data/checkpoint_data/sue_df_linked.pkl')
sue_df.columns = ['IBES_TICKER', 'OFTIC', 'MEASURE', 'FISCALP', 'PYEAR', 'PMON', 'USFIRM',
                 'ibes_anndate', 'actual', 'surpmean', 'surpstdev', 'suescore', 'PERMNO',
                 'pyear_month_date', 'bias', 'date', 'SHRCD', 'SICCD', 'NUMEST_sue']
sue_df['2d_sic'] = sue_df['SICCD'].apply(lambda x: get_2d_sic(x))
sue_df = sue_df.sort_values(['pyear_month_date']).drop_duplicates(['PERMNO', 'ibes_anndate'], keep='last')

In [3]:
## read cleaned returns file
ret_df = pd.read_pickle('data/checkpoint_data/crsp_dailydata_holding_w_period_returns.pkl')
ret_df[['RET', '30d_ret', '60d_ret', '2d_ret', '3d_ret','back_1d', 'back_5d', 'back_30d']] = 100 * ret_df[['RET', '30d_ret', '60d_ret', '2d_ret', '3d_ret','back_1d', 'back_5d', 'back_30d']]
ret_df.columns = ['ret_date', 'PERMNO', 'TICKER', 'PRC', 'RET', '60d_ret', '2d_ret', 
                  '3d_ret', 'back_1d_ret', 'back_5d_ret', 'back_30d_ret', 'lagged_PRC', '30d_ret']
ret_df['back_30d_to_60d_ret'] = ((1 + ret_df['back_30d_ret']/100) * (1 + ret_df['60d_ret']/100) - 1) * 100
ret_df['back_30d_to_30d_ret'] = ((1 + ret_df['back_30d_ret']/100) * (1 + ret_df['30d_ret']/100) - 1) * 100

In [4]:
## read in 10yr yield data
tenyr_df = pd.read_csv('data/ten_yr_yields.csv', parse_dates=['DATE'])
tenyr_df['DGS10'] = tenyr_df['DGS10'].apply(lambda x: math.nan if x == '.' else float(x))
tenyr_df = tenyr_df.dropna()
tenyr_df.columns = ['tenyr_date', '10yT']

In [5]:
## read in 1m yield data
ff3_df = pd.read_csv('C:/Users/jackl/OneDrive/Documents/finance_research/sunderam/data/fama_french_3factor_copy.csv')
ff3_df['date'] = ff3_df['date'].apply(lambda x: pd.to_datetime(str(x)+'01', yearfirst=True))
ff3_df['month'], ff3_df['year'] = ff3_df['date'].dt.month, ff3_df['date'].dt.year
ff3_df = ff3_df[['Mkt-RF', 'RF', 'month', 'year']]
ff3_df.columns = ['Mkt-RF', '1mT', 'month_1m', 'year_1m']

In [6]:
## read in fundamental data for M/B
an_df = pd.read_csv('data/price_to_book.csv', parse_dates=['datadate'])
an_df = an_df[['GVKEY', 'cusip', 'LPERMNO', 'datadate', 'fyear', 'ceq', 'txditc', 'prcc_f', 'csho']]
an_df['be'] = (an_df['ceq'] + an_df['txditc']).apply(lambda x: math.nan if x < 0 else x)
an_df['me'] = an_df['prcc_f'] * an_df['csho']
an_df['mb'] = an_df['me']/an_df['be']
an_df['bm'] = an_df['be']/an_df['me']
an_df['LPERMNO'] = an_df['LPERMNO'].astype('Int64')
an_df['8dcusip'] = an_df['cusip'].apply(lambda x: x[:-1])
an_df = an_df.sort_values(['datadate']).drop_duplicates(['LPERMNO', 'fyear'], keep='last')

In [7]:
ltg_df = pd.read_pickle('data/checkpoint_data/ltg.pkl')

## Merge Data

In [8]:
## merge IBES file with returns data
full1_df = pd.merge(sue_df, 
                    ret_df, 
                    left_on=['ibes_anndate', 'PERMNO'], 
                    right_on=['ret_date', 'PERMNO'], 
                    how='left')

del sue_df
del ret_df

MemoryError: Unable to allocate 329. MiB for an array with shape (1, 43065497) and data type float64

In [ ]:
## define dE/P
full1_df['de_P'] = (full1_df['actual'] - full1_df['surpmean'])/full1_df['lagged_PRC']

In [ ]:
## get quarter and month indices
full1_df['CAL_QTR'] = full1_df['PMON'].apply(lambda x: get_cal_qtr(x))
full1_df['anndats_month'] = full1_df['ibes_anndate'].dt.month
full1_df['anndats_year'] = full1_df['ibes_anndate'].dt.year
full1_df['anndats_CAL_QTR'] = full1_df['anndats_month'].apply(lambda x: get_cal_qtr(x))

#get qtr index for the quarter the announcement date occurs
anndats_year_qtrs_tuples = list(tuple(zip(list(full1_df['anndats_year']), list(full1_df['anndats_CAL_QTR']))))
dict_year_qtrs_mapping = dict(zip(sorted(set(anndats_year_qtrs_tuples), key=lambda element: (element[0], element[1])), range(len(set(anndats_year_qtrs_tuples)))))
full1_df['qtr_index'] = [dict_year_qtrs_mapping[tuple_] for tuple_ in anndats_year_qtrs_tuples]

#get qtr index for the quarter the announcement is for
anndats_year_qtrs_tuples = list(tuple(zip(list(full1_df['PYEAR']), list(full1_df['CAL_QTR']))))
dict_year_qtrs_mapping = dict(zip(sorted(set(anndats_year_qtrs_tuples), key=lambda element: (element[0], element[1])), range(len(set(anndats_year_qtrs_tuples)))))
full1_df['actual_qtr_index'] = [dict_year_qtrs_mapping[tuple_] for tuple_ in anndats_year_qtrs_tuples]

#get month index for the announcement date
anndats_year_month_tuples = list(tuple(zip(list(full1_df['anndats_year']), list(full1_df['anndats_month']))))
dict_year_month_mapping = dict(zip(sorted(set(anndats_year_month_tuples), key=lambda element: (element[0], element[1])), range(len(set(anndats_year_month_tuples)))))
full1_df['month_index'] = [dict_year_month_mapping[tuple_] for tuple_ in anndats_year_month_tuples]

#get four-year interval index
full1_df['interval_index'] = full1_df['anndats_year'].apply(lambda x: math.floor((x - 1992)/4))

In [ ]:
## merge with LTG data
full1_df = pd.merge(full1_df, 
                    ltg_df, 
                    left_on=['IBES_TICKER', 'anndats_month', 'anndats_year'], 
                    right_on=['TICKER', 'STATPERS_month', 'STATPERS_year'], 
                    how='left')

In [ ]:
## merge with interest rate data
full1_df = pd.merge(full1_df, 
                    ff3_df, 
                    left_on=['anndats_month', 'anndats_year'], 
                    right_on=['month_1m', 'year_1m'])

full1_df = pd.merge_asof(full1_df.sort_values('ibes_anndate'), 
                         tenyr_df, 
                         left_on=['ibes_anndate'], 
                         right_on=['tenyr_date'], 
                         direction='forward')

In [ ]:
## merge with M/B data
full1_df = pd.merge_asof(full1_df.sort_values('ibes_anndate'), 
                          an_df.sort_values('datadate'), 
                          left_by='PERMNO', 
                          right_by='LPERMNO', 
                          left_on='ibes_anndate', 
                          right_on='datadate', 
                          direction='backward', 
                          tolerance=pd.Timedelta(days=365))

In [ ]:
## set up earnings data
full1_df['log_earnings'] = np.log(full1_df['actual'] * full1_df['csho'])
full1_df['actual_qtr_index_match_1'] = full1_df['actual_qtr_index'] - 1
full1_df['actual_qtr_index_match_4'] = full1_df['actual_qtr_index'] - 4
earnings_df = full1_df[['PERMNO', 'actual_qtr_index', 'log_earnings']]
earnings_df.columns = ['PERMNO', 'match_index', 'log_earnings']

In [ ]:
## merge raw earnings data
full1_df = pd.merge(full1_df,  
                    earnings_df, 
                    left_on=['PERMNO', 'actual_qtr_index_match_1'], 
                    right_on=['PERMNO', 'match_index'], 
                    how='left', 
                    suffixes=['', '_lag1']).drop(columns=['match_index', 'actual_qtr_index_match_1'])
full1_df['diff_log_earnings'] = full1_df['log_earnings'] - full1_df['log_earnings_lag1']

earnings_df = full1_df[['PERMNO', 'actual_qtr_index', 'log_earnings', 'diff_log_earnings']]
earnings_df.columns = ['PERMNO', 'match_index', 'log_earnings', 'diff_log_earnings']
full1_df = pd.merge(full1_df,  
                    earnings_df, 
                    left_on=['PERMNO', 'actual_qtr_index_match_4'], 
                    right_on=['PERMNO', 'match_index'], 
                    how='left', 
                    suffixes=['', '_lag4']).drop(columns=['match_index', 'actual_qtr_index_match_4'])

## Trim Data and Export

In [ ]:
## trimmed data starts here
full1_df_trimmed = full1_df.copy()

#replace infs with nans
full1_df_trimmed.replace([np.inf, -np.inf], np.nan, inplace=True)

#drop duplicates where a company has more than one observation for the same quarter being predicted
full1_df_trimmed = full1_df_trimmed.drop_duplicates(subset=['pyear_month_date', 'PERMNO'])

percentile_trim_vars = ['30d_ret',
                        '60d_ret', 
                        '2d_ret', 
                        'back_30d_to_60d_ret',
                        'back_30d_to_30d_ret',
                        'back_1d_ret', 
                        'back_5d_ret', 
                        'back_30d_ret', 
                        'mb', 
                        'bm', 
                        'log_earnings', 
                        'log_earnings_lag4', 
                        'diff_log_earnings', 
                        'diff_log_earnings_lag4'
] 

bouchaud_cols = ['suescore', 
                'de_P', 
                'MEDEST', 
                'MEANEST',
                'MEDEST_p1', 
                'MEANEST_p1',
                'MEDEST_p2', 
                'MEANEST_p2', 
                'MEDEST_l1', 
                'MEANEST_l1', 
                'chg_MEDEST_2', 
                'chg_MEANEST_2', 
                'chg_MEDEST_3', 
                'chg_MEANEST_3', 
                'chg_MEANEST_rev31',
                'chg_MEDEST_rev31',
                'chg_MEANEST_rev32',
                'chg_MEDEST_rev32'
]

#trim 1%/99% conditional on quarter
full1_df_trimmed = get_truncated_df(full1_df_trimmed, 
                                    percentile_trim_vars,
                                    year_column_name='qtr_index', 
                                    low=0.01, 
                                    high=0.99)

full1_df_trimmed = trim_bouchaud_cols(bouchaud_cols, 
                                      full1_df_trimmed)

In [ ]:
## Summary Stats
sum_stats_vars = percentile_trim_vars + bouchaud_cols
sum_stats = full1_df_trimmed[sum_stats_vars].describe(percentiles=[.25, .5, .75])

In [ ]:
ff_match = pd.read_csv('C:/Users/jackl/OneDrive/Documents/finance_research/ec980/seo_data/raw_data/SIC_to_Fama_French_industry.csv')
full1_df_trimmed = pd.merge(full1_df_trimmed, 
                            ff_match, 
                            left_on='SICCD', 
                            right_on='SIC', 
                            how='left'
).drop(columns=['SIC', 'SIC0'])

#get deciles for various variables
panel_df = full1_df_trimmed.copy()
ff_12_list = list(set(panel_df['FF_12'].dropna()))
total_dfs = []
for ff_12 in tqdm(ff_12_list): 
    panel_subset_df = panel_df[panel_df['FF_12']==ff_12]
    for qtr_index in range(panel_df['qtr_index'].max() + 1):
        panel_subset_df_qtr = panel_subset_df[panel_subset_df['qtr_index']==qtr_index]
        panel_subset_df_qtr['suescore_decile'] = pd.qcut(panel_subset_df_qtr['suescore'].rank(method='first'), 10, labels=False, duplicates='drop') + 1
        panel_subset_df_qtr['de_P_decile'] = pd.qcut(panel_subset_df_qtr['de_P'].rank(method='first'), 10, labels=False, duplicates='drop') + 1
        panel_subset_df_qtr['mb_decile'] = pd.qcut(panel_subset_df_qtr['mb'].rank(method='first'), 10, labels=False, duplicates='drop') + 1
        panel_subset_df_qtr['mb_indicator'] = pd.qcut(panel_subset_df_qtr['mb'].rank(method='first'), 2, labels=False, duplicates='drop')
        total_dfs.append(panel_subset_df_qtr)
full1_df_trimmed = pd.concat(total_dfs)

In [ ]:
full1_df_trimmed[['PERMNO', 'LPERMNO']] = full1_df_trimmed[['PERMNO', 'LPERMNO']].astype(float)
full1_df_trimmed.to_stata('data/checkpoint_data/sue_ret_df_trimmed_year_cond.dta')

## Create M/B on LTG Dataset

In [20]:
## read in fundamental data for M/B
an_df = pd.read_csv('data/price_to_book.csv', parse_dates=['datadate'])
an_df = an_df[['GVKEY', 'cusip', 'LPERMNO', 'datadate', 'fyear', 'ceq', 'txditc', 'prcc_f', 'csho']]
an_df['be'] = (an_df['ceq'] + an_df['txditc']).apply(lambda x: math.nan if x < 0 else x)
an_df['me'] = an_df['prcc_f'] * an_df['csho']
an_df['mb'] = an_df['me']/an_df['be']
an_df['bm'] = an_df['be']/an_df['me']
an_df['LPERMNO'] = an_df['LPERMNO'].astype('Int64')
an_df['8dcusip'] = an_df['cusip'].apply(lambda x: x[:-1])
an_df = an_df.sort_values(['datadate']).drop_duplicates(['LPERMNO', 'fyear'], keep='last')

ltg_df = pd.read_pickle('data/checkpoint_data/ltg.pkl')

In [21]:
ltg_mb_df = pd.merge_asof(an_df.sort_values('datadate'), 
                          ltg_df.sort_values('STATPERS'), 
                          left_by='8dcusip', 
                          right_by='CUSIP', 
                          left_on='datadate', 
                          right_on='STATPERS', 
                          direction='backward', 
                          tolerance=pd.Timedelta(days=31))

In [22]:
ltg_mb_df = ltg_mb_df.dropna(subset=['fyear'])

In [23]:
ltg_mb_df['interval_index'] = ltg_mb_df['fyear'].apply(lambda x: math.floor((x - 1992)/4))

In [24]:
ltg_mb_df = get_truncated_df(ltg_mb_df, 
                             ['MEDEST', 'MEANEST','MEDEST_p1', 'MEANEST_p1','MEDEST_p2', 
                              'MEANEST_p2', 'MEDEST_l1', 'MEANEST_l1', 'chg_MEDEST_2', 
                              'chg_MEANEST_2', 'chg_MEDEST_3', 'chg_MEANEST_3', 'mb', 'bm'], 
                              year_column_name='fyear', 
                              low=0.01, 
                              high=0.99)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 10.22it/s]


In [25]:
ltg_mb_df['mb_trimmed_strict'] = ltg_mb_df['mb'].apply(lambda x: trim_strict(x, -math.inf, 10))
ltg_mb_df['MEANEST_trimmed_strict'] = ltg_mb_df['MEANEST'].apply(lambda x: trim_strict(x, -30, 50))

In [26]:
ltg_mb_df[['LPERMNO']] = ltg_mb_df[['LPERMNO']].astype(float)
ltg_mb_df.to_stata('data/checkpoint_data/mb_ltg.dta')

c:\users\jackl\appdata\local\programs\python\python39\lib\site-packages\pandas\io\stata.py:2346: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    8dcusip   ->   _8dcusip

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)
